In [13]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

#!pip install huggingface_hub
#!pip install ipywidgets
#!pip install accelerate bitsandbytes
#!pip install -U transformers 

from huggingface_hub import notebook_login

import json
from tqdm import tqdm

In [14]:
notebook_login()

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", quantization_config=quantization_config)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
INPUT_PTH= "C:/Users/Allie/Documents/GitHub/policy-classifier/policy_scraping/outputs/scraped_pdfs.json"

with open(INPUT_PTH,"r", encoding="utf-8") as f:
    pdf_json = json.load(f)

In [18]:
#print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 10           |        cudaMalloc retries: 14        |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   7474 MiB |  29214 MiB |   1428 GiB |   1421 GiB |
|       from large pool |   7434 MiB |  29191 MiB |   1426 GiB |   1419 GiB |
|       from small pool |     39 MiB |     87 MiB |      2 GiB |      1 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   7474 MiB |  29214 MiB |   1428 GiB |   1421 GiB |
|       from large pool |   7434 MiB |  29191 MiB |   1426 GiB |

In [19]:
#OutOfMemoryError: CUDA out of memory. Tried to allocate 16.36 GiB. 
#GPU 0 has a total capacty of 16.00 GiB of which 0 bytes is free. 
#Of the allocated memory 24.31 GiB is allocated by PyTorch, 
#and 1.56 GiB is reserved by PyTorch but unallocated. 
#If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  
#See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

#https://pytorch.org/docs/stable/notes/faq.html

indexs = tokenizer("X Yes No", return_tensors="pt")

yes_idx = indexs["input_ids"][0,2]
no_idx = indexs["input_ids"][0,3]

for hash in tqdm(pdf_json):
  inputs = tokenizer(f"""{pdf_json[hash]["text"]}
  ---
  Is this policy about restoring the environment? Answer Yes or No:
""", return_tensors="pt", return_attention_mask=False)

  outputs = model.generate(**inputs, max_new_tokens=5, min_new_tokens=1, output_scores=True, return_dict_in_generate=True)
  text = tokenizer.batch_decode(outputs["sequences"])[0]
  print(text)
  print(outputs["scores"][0][0,yes_idx])
  print(outputs["scores"][0][0,no_idx])
  #



  1%|          | 1/164 [00:19<53:43, 19.77s/it]

<bos>Public Service Agreement 2010 -2014 (Croke Park Agreement) Integrated Action Plan for the Department of Transport, Tourism and Sport and its agencies Updated: October 2012 1. Better human resource management To include , for example, actions around t he reduction of staff numbers; the redeployment of staff to areas of greatest need; the restructuring/re configuration of service delivery ; changes to work practices; revisi ons in attendance arrangements; absence management ; performance management etc. Terms of the Public Service Agreement 2010 – 2014 Action /commitment Target Date for Implementation 15 Public Service Numbers: Reduce staff complement in Department from 514 (2011) to 495 by end 2012, and from 1044 to 982 in the non -commercial agencies, and annually thereafter to 2015 in line with targets specified by Department of Public Expenditure and Reform. Update: Proposals from Fáilte Ireland for voluntary redundancy and other exit mechanisms to fast -track the potential savi

  1%|          | 1/164 [00:22<1:01:57, 22.81s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.36 GiB. GPU 0 has a total capacty of 16.00 GiB of which 0 bytes is free. Of the allocated memory 10.17 GiB is allocated by PyTorch, and 16.47 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
texts = ["""Saccharomyces cerevisiae has been used for at least eight millennia in the production of alcoholic beverages (41). Along with ethanol and carbon dioxide, fermenting cultures of this yeast produce many low-molecular-weight flavor compounds. These alcohols, aldehydes, organic acids, esters, organic sulfides, and carbonyl compounds have a strong impact on product quality. Indeed, the subtle aroma balance of these compounds in fermented foods and beverages is often used as an organoleptic fingerprint for specific products and brands (42). Food fermentation by yeast and lactic acid bacteria is accompanied by the formation of the aliphatic and aromatic alcohols known as fusel alcohols. Fusel oil, which derives its name from the German word fusel (bad liquor), is obtained during the distillation of spirits and is enriched with these higher alcohols. While fusel alcohols at high concentrations impart off-flavors, low concentrations of these compounds and their esters make an essential contribution to the flavors and aromas of fermented foods and beverages. Fusel alcohols are derived from amino acid catabolism via a pathway that was first proposed a century ago by Ehrlich (13). Amino acids represent the major source of the assimilable nitrogen in wort and grape must, and these amino acids are taken up by yeast in a sequential manner (23, 32). Amino acids that are assimilated by the Ehrlich pathway (valine, leucine, isoleucine, methionine, and phenylalanine) are taken up slowly throughout the fermentation time (32). After the initial transamination reaction (Fig. ​(Fig.1),1), the resulting α-keto acid cannot be redirected into central carbon metabolism. Before α-keto acids are excreted into the growth medium, yeast cells convert them into fusel alcohols or acids via the Ehrlich pathway. FIG. 1. The Ehrlich pathway. Catabolism of branched-chain amino acids (leucine, valine, and isoleucine), aromatic amino acids (phenylalanine, tyrosine, and trytophan), and the sulfur-containing amino acid (methionine) leads to the formation of fusel acids and ... Current scientific interest in the Ehrlich pathway is supported by increased demands for natural flavor compounds such as isoamyl alcohol and 2-phenylethanol, which can be produced from amino acids in yeast-based bioconversion processes (14), as well as by the need to control flavor profiles of fermented food products. The goal of this paper is to present a concise centenary overview of the biochemistry, molecular biology, and physiology of this important pathway in S.""",
"""Sense induction seeks to automatically identify word senses
directly from a corpus. A key assumption underlying previous work is that the context
surrounding an ambiguous word is indicative of its meaning. Sense induction is thus
typically viewed as an unsupervised clustering problem where the aim is to partition a
 word's contexts into different classes, each representing a word sense. Our work places
  sense induction in a Bayesian context by modeling the contexts of the ambiguous word as
  samples from a multinomial distribution over senses which are in turn characterized
  as distributions over words. The Bayesian framework provides a principled way
  to incorporate a wide range of features beyond lexical co-occurrences and to
  systematically assess their utility on the sense induction task. The proposed
  approach yields improvements over state-of-the-art systems on a benchmark dataset"""]

indexs = tokenizer("X Yes No", return_tensors="pt")


yes_idx = indexs["input_ids"][0,2]
no_idx = indexs["input_ids"][0,3]

for text in texts:
  inputs = tokenizer(f"""{text}
  ---
  Is this article about inducing the number of senses a word would have in a dictionary? Answer Yes or No:
""", return_tensors="pt", return_attention_mask=False)

  outputs = model.generate(**inputs, max_new_tokens=5, min_new_tokens=1, output_scores=True, return_dict_in_generate=True)
  text = tokenizer.batch_decode(outputs["sequences"])[0]
  print(text)
  print(outputs["scores"][0][0,yes_idx])
  print(outputs["scores"][0][0,no_idx])